In [1]:
import pandas
import math
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy.stats import norm
from statsmodels.sandbox.regression.gmm import GMM
from statsmodels.base.model import GenericLikelihoodModel
from scipy.stats import genextreme

In [8]:
#load data into memory
data1 = np.genfromtxt('OTC_Data_forStata_modified.csv', delimiter=',')

#partition correctly
y = data1[5]
x = sm.add_constant(data1[6])

In [9]:
part_a = sm.OLS(y,x).fit()
print(part_a.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 1.182e+06
Date:                Thu, 21 Mar 2024   Prob (F-statistic):           1.27e-33
Time:                        18:37:02   Log-Likelihood:                -64.064
No. Observations:                  15   AIC:                             132.1
Df Residuals:                      13   BIC:                             133.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.6083      4.975      0.926      0.3

/Users/ynbsztl/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=15
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [13]:
class part_b(GenericLikelihoodModel):

    def nloglikeobs(self, params):
        t1, t2, sigma = params
        endog, exog = self.endog, self.exog.squeeze()
        eps = endog - t1 - t2*exog
        return - genextreme.logpdf(eps, c=0, loc=0, scale=sigma).sum()
    
    def fit(self, start_params=None, maxiter=10000, maxfun=5000, **kwds):
        if start_params == None:
            start_params = start_params = [.5, .5,.5]
        return super(part_b, self).fit(start_params=start_params,
                                       maxiter=maxiter, maxfun=maxfun, **kwds)

model_b = part_b(data1[5],data1[6])

result_b = model_b.fit()
print(result_b.summary(xname=['theta_1', 'theta_2', 'sigma']))


Optimization terminated successfully.
         Current function value: 3.531275
         Iterations: 232
         Function evaluations: 420
                                part_b Results                                
Dep. Variable:                      y   Log-Likelihood:                -52.969
Model:                         part_b   AIC:                             105.9
Method:            Maximum Likelihood   BIC:                             105.9
Date:                Thu, 21 Mar 2024                                         
Time:                        18:42:24                                         
No. Observations:                  15                                         
Df Residuals:                      14                                         
Df Model:                           0                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------

/Users/ynbsztl/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:2858: RuntimeWarning: overflow encountered in exp
  pex2 = np.exp(logpex2)
/Users/ynbsztl/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:2694: UserWarning: df_model + k_constant differs from nparams
  warnings.warn("df_model + k_constant differs from nparams")
/Users/ynbsztl/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:2696: UserWarning: df_resid differs from nobs - nparams
  warnings.warn("df_resid differs from nobs - nparams")


In [ ]:
#part c - GMM

class part_c(GMM):
    """class for evaluating question 1 part c"""
    
    def __init__(self, *args, **kwds):
        # set appropriate counts for moment conditions and parameters
        kwds.setdefault('k_moms', 2)
        kwds.setdefault('k_params',2)
        super(part_c, self).__init__(*args, **kwds)
    
    
    def fit(self, start_params=None, maxiter=10000, **kwds):
        if start_params == None:
            start_params = np.array([.5, .5])
        return super(part_c, self).fit(start_params=start_params,
                                       maxiter=maxiter,  **kwds)
    
    
    def momcond(self, params):
        t1,t2 = params  #unwrap parameters
        endog, exog = self.endog, self.exog.squeeze()
        eps = endog - t1 - t2*exog 
        g = np.column_stack( (eps, eps*exog ))
        return g 

    
model_c = part_c(data1[0],data1[1], None)
result_c = model_c.fit(maxiter=2, optim_method='nm', wargs=dict(centered=False))
print(result_c.summary(xname=['theta_1', 'theta_2']))


#sources:
#https://github.com/josef-pkt/misc/blob/master/notebooks/ex_gmm_gamma.ipynb
#https://www.statsmodels.org/dev/generated/statsmodels.sandbox.regression.gmm.GMM.html#statsmodels.sandbox.regression.gmm.GMM
#https://gist.github.com/josef-pkt/6895915

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 37
         Function evaluations: 70
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 13
         Function evaluations: 26
                                part_c Results                                
Dep. Variable:                      y   Hansen J:                    1.288e-06
Model:                         part_c   Prob (Hansen J):                   nan
Method:                           GMM                                         
Date:                Wed, 20 Mar 2024                                         
Time:                        11:55:48                                         
No. Observations:                 500                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
theta_1      